## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/Ecommerce_Customers.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)

In [3]:
# Create a view or table

temp_table_name = "Ecommerce_Customers_csv"

df.createOrReplaceTempView(temp_table_name)

In [4]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "Ecommerce_Customers_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [5]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
df.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [7]:
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 5 rows

In [8]:
df.columns

Out[41]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')

In [10]:
output = assembler.transform(df)

In [11]:
output.head(1)

Out[44]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [12]:
output.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
summary| Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
 count| 500| 500| 500| 500| 500| 500| 500| 500|
 mean| null| null| null| 33.05319351819619|12.052487937166134| 37.06044542094859| 3.533461555915055| 499.3140382585909|
 stddev| null| null| null|0.9925631110845354|0.9942156084725424|1.0104889067564033| 0.9992775024112585| 79.3147815497068|
 min|aaron04@yahoo.com|0001 Mack MillNor...| AliceBlue|29.532428967057943| 8.508152176032603| 33.91384724758464| 0.2699010899842742| 256.67058229005585|
 max|zscott@wright.com|Unit 7502 Box 834...|YellowGreen| 36.13966248879052|15.126994288792467|40.005181638101895| 6.922689335035808| 765.5184619388373|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+

In [13]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [14]:
final_data = output.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show(10)

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
+--------------------+-------------------+
only showing top 10 rows

In [16]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 346|
 mean| 498.57071422784134|
 stddev| 78.58731922957459|
 min| 256.67058229005585|
 max| 725.5848140556806|
+-------+-------------------+

In [18]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 154|
 mean| 500.9841039380665|
 stddev| 81.15978047919299|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [19]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show(10)

+--------------------+
 residuals|
+--------------------+
 9.286622761049102|
 -11.76058718280649|
 -13.13217031686139|
-0.07071988025893461|
 3.0848149540505574|
 4.471478409432734|
-0.18389674914124043|
 -8.949785950566252|
 -5.63359454915917|
 -5.176947437393267|
+--------------------+
only showing top 10 rows

In [23]:
test_results.rootMeanSquaredError

Out[56]: 10.296055766068301

In [24]:
test_results.r2

Out[57]: 0.9838009578685075

In [25]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [26]:
unlabled_data = test_data.select('features')

In [27]:
pred = lr_model.transform(unlabled_data)

In [28]:
pred.show(10)

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 399.3537283115784|
[30.3931845423455...| 331.6894569860001|
[31.0662181616375...|462.06546352453574|
[31.2606468698795...| 421.3973511372103|
[31.3091926408918...|429.63590288588307|
[31.3584771924370...| 490.7044720400427|
[31.3895854806643...|410.25350780912413|
[31.4474464941278...|427.55252804579027|
[31.5147378578019...| 495.4460825456206|
[31.5171218025062...| 281.095368087779|
+--------------------+------------------+
only showing top 10 rows